In [1]:
import open_clip
import numpy as np
import torch
from openclip_encoder import OpenCLIPNetwork

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# model, _, preprocess = open_clip.create_model_and_transforms('convnext_base_w', pretrained='laion2b_s13b_b82k_augreg')
clip_model_type = "ViT-B-16"
clip_model_pretrained = 'laion2b_s34b_b88k'
clip_n_dims = 512
model, _, _ = open_clip.create_model_and_transforms(
            clip_model_type,
            pretrained=clip_model_pretrained,
            precision="fp16",
        )
# model.eval()

In [4]:
tokenizer = open_clip.get_tokenizer(clip_model_type)
model = model.to("cuda")


In [5]:
positives = ("switch", "Nintendo", "red", "controller", 
             "UNO", "card", "stack"
             "sofa","grey",
             "Gundam", 
             "Pikachu", 
             "Xbox", "wireless")
with torch.no_grad():
    tok_phrases = torch.cat([tokenizer(phrase) for phrase in positives]).to(device)
    pos_embeds = model.encode_text(tok_phrases)
pos_embeds /= pos_embeds.norm(dim=-1, keepdim=True)  

In [6]:
pos_embeds.shape


torch.Size([12, 512])

In [7]:
# now import features pretrained preprocessed npy
import sys
import types
sys.path.append("~/LangSplat")

dict_args = {}
args = types.SimpleNamespace(**dict_args)
# parser.add_argument("--iteration", default=-1, type=int)
# parser.add_argument("--skip_train", action="store_true")
# parser.add_argument("--skip_test", action="store_true")
# parser.add_argument("--quiet", action="store_true")
# parser.add_argument("--include_feature", action="store_true")
# parser.add_argument("--model_path", default = "/home/sarah/LangSplat/sofa/sofa_1")
args.iteration = -1
args.skip_train = True
args.skip_test = True
args.quiet = True
args.include_feature = True
args.model_path = "/home/sarah/LangSplat/sofa/sofa_1"

In [11]:
sys.path.append("/home/sarah/LangSplat/")
sys.path.append("/home/sarah/LangSplat/utils/")
from general_utils import safe_state
from render import render_sets
sys.path.append("/home/sarah/LangSplat/scene")
from scene import Scene
render_sets(model.extract(args), args.iteration, pipeline.extract(args), args.skip_train, args.skip_test, args)

ModuleNotFoundError: No module named 'utils.system_utils'; 'utils' is not a package

In [26]:
!cd /home/sarah/LangSplat/Scene && pip install -e .

Obtaining file:///home/sarah/LangSplat
ERROR: file:///home/sarah/LangSplat does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [35]:
from arguments import ModelParams, PipelineParams, get_combined_args
# from gaussian_renderer import GaussianModel
from scene import Scene
sys.path.append('/home/sarah/LangSplat/utils')

with torch.no_grad():
#     gaussians = GaussianModel(dataset.sh_degree)
    scene = Scene(dataset, gaussians, shuffle=False)
    checkpoint = os.path.join(args.model_path, 'chkpnt30000.pth')
    (model_params, first_iter) = torch.load(checkpoint)
    gaussians.restore(model_params, args, mode='test')

    bg_color = [1,1,1] if dataset.white_background else [0, 0, 0]
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

ModuleNotFoundError: No module named 'utils.system_utils'; 'utils' is not a package

In [30]:


def render_set(model_path, source_path, name, iteration, views, gaussians, pipeline, background, args):
    render_path = os.path.join(model_path, name, "ours_{}".format(iteration), "renders")
    gts_path = os.path.join(model_path, name, "ours_{}".format(iteration), "gt")
    render_npy_path = os.path.join(model_path, name, "ours_{}".format(iteration), "renders_npy")
    gts_npy_path = os.path.join(model_path, name, "ours_{}".format(iteration), "gt_npy")

    makedirs(render_npy_path, exist_ok=True)
    makedirs(gts_npy_path, exist_ok=True)
    makedirs(render_path, exist_ok=True)
    makedirs(gts_path, exist_ok=True)

    for idx, view in enumerate(tqdm(views, desc="Rendering progress")):
        output = render(view, gaussians, pipeline, background, args)

        if not args.include_feature:
            rendering = output["render"]
        else:
            rendering = output["language_feature_image"]
            
        if not args.include_feature:
            gt = view.original_image[0:3, :, :]
            
        else:
            gt, mask = view.get_language_feature(os.path.join(source_path, args.language_features_name), feature_level=args.feature_level)

        np.save(os.path.join(render_npy_path, '{0:05d}'.format(idx) + ".npy"),rendering.permute(1,2,0).cpu().numpy())
        np.save(os.path.join(gts_npy_path, '{0:05d}'.format(idx) + ".npy"),gt.permute(1,2,0).cpu().numpy())
        torchvision.utils.save_image(rendering, os.path.join(render_path, '{0:05d}'.format(idx) + ".png"))
        torchvision.utils.save_image(gt, os.path.join(gts_path, '{0:05d}'.format(idx) + ".png"))
               
def render_sets(dataset : ModelParams, iteration : int, pipeline : PipelineParams, skip_train : bool, skip_test : bool, args):
    with torch.no_grad():
        gaussians = GaussianModel(dataset.sh_degree)
        scene = Scene(dataset, gaussians, shuffle=False)
        checkpoint = os.path.join(args.model_path, 'chkpnt30000.pth')
        (model_params, first_iter) = torch.load(checkpoint)
        gaussians.restore(model_params, args, mode='test')
        
        bg_color = [1,1,1] if dataset.white_background else [0, 0, 0]
        background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

        if not skip_train:
             render_set(dataset.model_path, dataset.source_path, "train", scene.loaded_iter, scene.getTrainCameras(), gaussians, pipeline, background, args)

        if not skip_test:
             render_set(dataset.model_path, dataset.source_path, "test", scene.loaded_iter, scene.getTestCameras(), gaussians, pipeline, background, args)

ModuleNotFoundError: No module named 'utils.system_utils'; 'utils' is not a package